In [339]:
def manu_entry(start_month):
    #for month in range(1,13):
    start_year = '108'
    start_month = str(start_month)
    start_day = '1'
    end_year = start_year
    end_month = start_month
    end_day = '31'
    cause_of_action = '公共危險'
    main_content = '酒精濃度'
    #<input type="text" id="dy1" maxlength="3" size="3" name="dy1" title="開始日期的年度">
    driver.find_element_by_name('dy1').send_keys( start_year )
    driver.find_element_by_name('dm1').send_keys( start_month )
    driver.find_element_by_name('dd1').send_keys( start_day )
    driver.find_element_by_name('dy2').send_keys( end_year )
    driver.find_element_by_name('dm2').send_keys( end_month )
    driver.find_element_by_name('dd2').send_keys( end_day )
    driver.find_element_by_name("jud_title").send_keys( cause_of_action )
    driver.find_element_by_name("jud_jmain").send_keys( main_content )
    #<input type="submit" name="ctl00$cp_content$btnQry" value="送出查詢" id="btnQry" class="btn btn-success">
    driver.find_element_by_name("ctl00$cp_content$btnQry").click()


In [446]:
def get_case_soup(case,gc):
    global case_url
    case_url = 'https://law.judicial.gov.tw/FJUD/' + url_part1 + str(case) + url_part2 + gc + url_part3
    user_agent = UserAgent().random
    headers = {'user-agent': user_agent}
    try:
        r = requests.get(case_url, headers=headers)
    except:
        print('error',case_url)
        time.sleep(60)
        r = requests.get(case_url, headers=headers)
    waittime = random.randint(3,7)
    time.sleep(waittime)
    case_soup = BeautifulSoup(r.text)
    return case_soup

In [445]:
def save_case(case_soup):  
    #檔名
    title = case_soup.find_all(class_="col-td")[0].text.replace(' ','').replace('\r\n','')
    date = case_soup.find_all(class_="col-td")[1].text.replace(' ','').replace('\r\n','')
    location = title.split('地')[0][2:]
    items =  title.split('年')[1].split('號')[0].split('第')
    process,num = items[0][:-1],items[1]
    yyymmdd = '-'.join( date[:-1].split('月')[0][2:].split('年') + [ date[:-1].split('月')[1] ] )
    filename = '_'.join( (location,yyymmdd,process,num) )
    #判決書全文
    content = case_soup.find(class_="text-pre text-pre-in").text
    #save
    path = 'D:/Google雲端硬碟/NCKU/法經/法經報告/判決書/'
    folder_name = location
    folder_path = path + folder_name
    folder = os.path.exists(folder_path)
    if not folder:
        os.makedirs(folder_path)
    file_path = folder_path +'/'+filename+'.txt'
    with open(file_path,'w',encoding='utf-8-sig') as w:
        w.writelines(content)

# 進入目錄網頁 run_1

In [9]:
from selenium import webdriver
driver = webdriver.Chrome('D:/chromedriver.exe')
driver.get('https://law.judicial.gov.tw/FJUD/default_AD.aspx')

In [ ]:
manu_entry(start_month)

# run_2
同個月分裡網址中的亂碼會一樣
輸入檢索後，進去iframe-data抓出第一個判決書網址 =case_url
不同地方法院下只會改變 gc= XXX
將case_url中gc=XXX改掉即可換法院
需要獲得:
各法院的gc代號,共幾筆


In [423]:
#driver.refresh()

#driver.back()
driver.switch_to.parent_frame()
#taipei = '//*[@id="collapseGrpCourt"]/div/ul/li[5]/a'
#driver.find_element_by_xpath(taipei).click()
#driver.switch_to.frame("iframe-data")

In [425]:
#1
soup_panel = BeautifulSoup(driver.page_source, 'lxml') #裝有地方法院、gc號、筆數的資訊
pattern = re.compile(r'\d+') #字串中篩選出數字的模式

In [427]:
#2
#取得第一個判決書網址
taipei = '//*[@id="collapseGrpCourt"]/div/ul/li[4]/a' #點擊列表第4個法院
driver.find_element_by_xpath(taipei).click()
#driver.find_element_by_xpath(taipei).click() #可能再一次
#driver.switch_to.frame("iframe-data") #轉至判決書列表

In [439]:
driver.switch_to.frame("iframe-data") #轉至判決書列表

In [447]:
#3
from bs4 import BeautifulSoup
import re
import requests
import os
import time
from fake_useragent import UserAgent
import random
#要進入臺北的判決書列表
soup = BeautifulSoup(driver.page_source, 'lxml')
if soup.find_all(class_ = "hlTitle_scroll") == []:
    driver.switch_to.frame("iframe-data")
href = soup.find_all(class_ = "hlTitle_scroll")[0].get('href')
url_part1 = href.split('ro=0')[0] +'ro='
url_part2 = href.split('ro=0')[1].split('gc=')[0] +'gc='
url_part3 = href.split('ro=0')[1].split('gc=')[1][3:]

empty = []
court_items = soup_panel.find(class_='panel-body').find_all('a')
for court in court_items:
    if '地方' not in court.text:
        continue
    num = int( pattern.search(court.text).group() ) #共num筆
    gc = court.get('data-groupid') #各地方法院的gc號
    print(court.text,gc)
    for case in range(num):
        case_soup = get_case_soup(case,gc)
        
        if case_soup.find(class_="main-section") ==None:
            save_case(case_soup)   
        elif case_soup.find(class_="main-section").h3.text == '查無資料':
            print('reset')
            driver.back()
            driver.back()
            driver.find_element_by_name("ctl00$cp_content$btnQry").click()
            taipei = '//*[@id="collapseGrpCourt"]/div/ul/li[4]/a'
            driver.find_element_by_xpath(taipei).click()
        
            save_case() 
        else:
            print(case_url)
            empty.append(case_url)
    driver.refresh()
    driver.find_element_by_xpath(taipei).click()
    time.sleep(150)
        

臺灣臺北地方法院216 TPD


KeyboardInterrupt: 

In [448]:
case_url

'https://law.judicial.gov.tw/FJUD/data.aspx?ro=3&q=7fc6fc576ab3aad07f46589f64b491ff&gy=jcourt&gc=TPD&sort=DS&ot=in'

In [440]:
soup = BeautifulSoup(driver.page_source, 'lxml')
soup

<html lang="zh-TW"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="NO-CACHE" http-equiv="CACHE-CONTROL"/>
<meta content="https://law.judicial.gov.twfblogo.png" property="og:image"/>
<title>
	查詢結果列表 &lt;i class="fa fa-angle-double-right"&gt;&lt;/i&gt;  臺灣臺北地方法院
</title><link href="../css/bootstrap.min.css" rel="stylesheet"/><link href="../css/font-awesome.min.css" rel="stylesheet"/><link href="../css/iframe.css" rel="stylesheet"/>
<script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script src="/js/jquery-3.2.1.min.js" type="text/javascript"></script>
<script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-100643133-1"></script>
<script>
        window.dataLayer = window.dataLayer || [];
        function gtag() { dataLayer.push(arguments); }
        gtag('js', new Date());

        gtag('config', 'UA-100643133-1');
    </script>
<me

# 中斷後再開
1540筆 3小時後被辨識碼中斷


In [282]:
driver.back()

In [ ]:
driver.find_element_by_name("ctl00$cp_content$btnQry").click()

In [ ]:
janu=['臺灣臺北地方法院153', '臺灣士林地方法院13', '臺灣新北地方法院278', '臺灣宜蘭地方法院84', '臺灣基隆地方法院55', '臺灣桃園地方法院314',
 '臺灣新竹地方法院82', '臺灣苗栗地方法院113', '臺灣臺中地方法院310', '臺灣彰化地方法院75', '臺灣南投地方法院77', '臺灣雲林地方法院77',
 '臺灣嘉義地方法院160', '臺灣臺南地方法院208', '臺灣高雄地方法院5', '臺灣橋頭地方法院4', '臺灣花蓮地方法院112', '臺灣臺東地方法院65',
 '臺灣屏東地方法院12', '福建金門地方法院1']
febr = ['臺灣臺北地方法院161','臺灣士林地方法院14','臺灣新北地方法院277','臺灣宜蘭地方法院78','臺灣基隆地方法院44','臺灣桃園地方法院258',
        '臺灣新竹地方法院39', '臺灣苗栗地方法院54', '臺灣臺中地方法院271', '臺灣彰化地方法院119', '臺灣南投地方法院75',
        '臺灣雲林地方法院76', '臺灣嘉義地方法院114', '臺灣臺南地方法院141', '臺灣高雄地方法院2', '臺灣橋頭地方法院3',
        '臺灣花蓮地方法院70', '臺灣臺東地方法院57', '臺灣屏東地方法院15']

In [352]:
gc_list = []
court_list = []
for court in court_items:
    if '地方' not in court.text:
        continue
    gc = court.get('data-groupid') #各地方法院的gc號
    print(court.text,gc)
    gc_list.append(gc)
    court_list.append(court.text)
print(court_list)


臺灣臺北地方法院161 TPD
臺灣士林地方法院14 SLD
臺灣新北地方法院277 PCD
臺灣宜蘭地方法院78 ILD
臺灣基隆地方法院44 KLD
臺灣桃園地方法院258 TYD
臺灣新竹地方法院39 SCD
臺灣苗栗地方法院54 MLD
臺灣臺中地方法院271 TCD
臺灣彰化地方法院119 CHD
臺灣南投地方法院75 NTD
臺灣雲林地方法院76 ULD
臺灣嘉義地方法院114 CYD
臺灣臺南地方法院141 TND
臺灣高雄地方法院2 KSD
臺灣橋頭地方法院3 CTD
臺灣花蓮地方法院70 HLD
臺灣臺東地方法院57 TTD
臺灣屏東地方法院15 PTD
['臺灣臺北地方法院161', '臺灣士林地方法院14', '臺灣新北地方法院277', '臺灣宜蘭地方法院78', '臺灣基隆地方法院44', '臺灣桃園地方法院258', '臺灣新竹地方法院39', '臺灣苗栗地方法院54', '臺灣臺中地方法院271', '臺灣彰化地方法院119', '臺灣南投地方法院75', '臺灣雲林地方法院76', '臺灣嘉義地方法院114', '臺灣臺南地方法院141', '臺灣高雄地方法院2', '臺灣橋頭地方法院3', '臺灣花蓮地方法院70', '臺灣臺東地方法院57', '臺灣屏東地方法院15']


## 被中斷時的縣市

In [449]:
gc = 'TPD'
start = 179
empty = []
for case in range(start,216):
    case_soup = get_case_soup(case,gc)

    if case_soup.find(class_="main-section") ==None:
        save_case(case_soup)   
    else:
        print(case_url)
        empty.append(case_url)

## 剩餘的縣市:  先run pop

In [451]:
find_location = '臺北'
pop_courts = []
for court in court_items:
    if '地方' not in court.text:
        continue
    pop_courts.append(court.text)
    if find_location in court.text:
        break
pop_courts

['臺灣臺北地方法院216']

In [452]:
empty = []
for court in soup_panel.find(class_='panel-body').find_all('a'):
    if '地方' not in court.text:
        continue
    #從蒐集不完整的縣市開始
    if court.text in pop_courts:
        print('pass',court.text)
        continue
    
    num = int( pattern.search(court.text).group() ) #共num筆
    gc = court.get('data-groupid') #各地方法院的gc號
    print(court.text,gc)
    for case in range(num):
        case_soup = get_case_soup(case,gc)
        
        if case_soup.find(class_="main-section") ==None:
            save_case(case_soup)   
        elif case_soup.find(class_="main-section").h3.text == '查無資料':
            print('reset')
            driver.back()
            driver.back()
            driver.find_element_by_name("ctl00$cp_content$btnQry").click()
            taipei = '//*[@id="collapseGrpCourt"]/div/ul/li[4]/a'
            driver.find_element_by_xpath(taipei).click()
        
            save_case() 
        else:
            print(case_url)
            empty.append(case_url)
    driver.refresh()
    driver.find_element_by_xpath(taipei).click()
    time.sleep(150)

pass 臺灣臺北地方法院216
臺灣士林地方法院27 SLD
臺灣新北地方法院345 PCD
臺灣宜蘭地方法院99 ILD


KeyboardInterrupt: 

# 缺個案

In [307]:
#driver.find_element_by_name("ctl00$cp_content$btnQry").click()
taipei = '//*[@id="collapseGrpCourt"]/div/ul/li[3]/a'
driver.find_element_by_xpath(taipei).click()

In [318]:
case_url = 'https://law.judicial.gov.tw/FJUD/data.aspx?ro=25&q=ed5ed0c9a9f8ecbcbeb92f5e3ff9ba73&gy=jcourt&gc=ULD&sort=DS&ot=in'
user_agent = UserAgent().random
headers = {'user-agent': user_agent}
r = requests.get(case_url, headers=headers)
case_soup = BeautifulSoup(r.text)
if case_soup.find(class_="main-section") ==None:
    save_case()
    '''
    elif case_soup.find(class_="main-section").h3.text == '查無資料':
        driver.back()
        driver.back()
        driver.find_element_by_name("ctl00$cp_content$btnQry").click()
        taipei = '//*[@id="collapseGrpCourt"]/div/ul/li[3]/a'
        driver.find_element_by_xpath(taipei).click()

        save_case() 
    '''
else:
    print(case_url)
    empty.append(case_url)

In [316]:
case_url

'https://law.judicial.gov.tw/FJUD/data.aspx?ro=20&q=ed5ed0c9a9f8ecbcbeb92f5e3ff9ba 73&gy=jcourt&gc=ULD&sort=DS&ot=in'

In [309]:
case_soup

<!DOCTYPE html>
<html lang="zh-TW">
<head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="NO-CACHE" http-equiv="CACHE-CONTROL"/>
<meta content="https://law.judicial.gov.twfblogo.png" property="og:image"/>
<title>
	系統訊息
</title><link href="../css/bootstrap.min.css" rel="stylesheet"/><link href="../css/font-awesome.min.css" rel="stylesheet"/><link href="../css/iframe.css" rel="stylesheet"/>
<script src="/js/jquery-3.2.1.min.js" type="text/javascript"></script>
<script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-100643133-1"></script>
<script>
        window.dataLayer = window.dataLayer || [];
        function gtag() { dataLayer.push(arguments); }
        gtag('js', new Date());

        gtag('config', 'UA-100643133-1');
    </script>
<meta content="noindex" name="robots"/></head>
<body>
<form action="../errorpage.aspx?frm=t&amp;err=L003" id="form1" method="post">
<div class="aspNetHi

# always if case_soup.find(class_="main-section") !=None: 

In [293]:
soup_error = case_soup

In [341]:
driver.back()
#manu_entry('1')
#driver.find_element_by_name("ctl00$cp_content$btnQry").click()
#driver.find_element_by_xpath(taipei).click()

In [292]:
case_soup.find(class_="main-section").h3.text

'查無資料'

In [300]:
case_soup.find(class_="main-section") ==None

True

# 檢查

In [357]:
#driver.forward()
driver.refresh()

In [358]:
BeautifulSoup(driver.page_source, 'lxml')

<html lang="zh-TW"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="NO-CACHE" http-equiv="CACHE-CONTROL"/><meta content="https://judicial-out.lawbank.com.tw/fblogo.png" property="og:image"/><title>
	司法院法學資料檢索系統
</title>
<!--[if lt IE 9]><script src="/js/html5shiv.min.js"></script><![endif]-->
<link href="../css/bootstrap.min.css" rel="stylesheet"/><link href="../css/font-awesome.min.css" rel="stylesheet"/><link href="../css/jquery.fancybox.css" media="screen" rel="stylesheet"/><link href="../css/layout.css" rel="stylesheet"/>
<script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script src="/js/jquery-3.2.1.min.js" type="text/javascript"></script>
<script src="/js/isMobile.js" type="text/javascript"></script>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-100643133-1"></s

In [350]:
case,case_soup.find_all(class_="col-td"),case_url,case_soup

(24,
 [<div class="col-td">
                      臺灣南投地方法院 108 年埔交簡字第 47 號刑事判決</div>,
  <div class="col-td">
                      民國 108 年 02 月 25 日</div>,
  <div class="col-td">
                      公共危險
                  </div>,
  <div class="col-td jud_content">
  <table border="0" width="100%">
  <tr>
  <td align="left" class="tab_content" nowrap="" valign="top">
  <div class="text-pre text-pre-in">臺灣南投地方法院刑事簡易判決　　　 108年度埔交簡字第47號
  聲　請　人　臺灣南投地方檢察署檢察官
  被　　　告　籃文松
  
  
  
  上列被告因公共危險案件，經檢察官聲請以簡易判決處刑（108
  年度偵字第649號），本院判決如下：
      主       文
  籃文松駕駛動力交通工具而有吐氣所含酒精濃度達每公升零點二
  五毫克以上，累犯，處有期徒刑伍月，如易科罰金以新臺幣壹仟
  元折算壹日。
      犯罪事實及理由
  一、犯罪事實：
      籃文松於民國108年1月24日20時30分許至同日22時許止，在
      其位於南投縣○○鎮○○路00號2樓現居地，飲用含有酒精
      成分之高粱酒，明知喝酒後在酒精未代謝完畢前不得駕駛動
      力交通工具，猶基於酒後駕車之犯意，於翌日（即1月25日
      ）6時許自上開居處駕駛車牌號碼00－000號營業大貨車欲至
      國姓鄉載貨而行駛於道路上；嗣行經國道6號高速公路西向
      34公里埔里交流道處時，為執行勤務之員警攔查後發現其身
      有酒味，顯有飲酒現象，乃於同日6時37分許當場對其施以
      呼氣酒精濃度測試，測得其呼氣中酒精濃度值為每公升0.57
      毫克，超過法律規定之標準，而查悉上情。
  二、認定犯罪事實所憑之證據及

## 看該月份檢索下的法院列表

In [154]:
#soup.find(class_= 'panel-body').a.get('data-groupid')
import re
pattern = re.compile(r'\d+')     
for i,place in enumerate( soup_panel.find(class_= 'panel-body').find_all('a') ):
    m = pattern.search(place.text) 
    if '地方' in place.text:
        print(i ,place.text , 'n=',pattern.search(place.text).group() ,'gc=',place.get('data-groupid')) 

2 臺灣臺北地方法院153 n= 153 gc= TPD
3 臺灣士林地方法院13 n= 13 gc= SLD
4 臺灣新北地方法院278 n= 278 gc= PCD
5 臺灣宜蘭地方法院84 n= 84 gc= ILD
6 臺灣基隆地方法院55 n= 55 gc= KLD
7 臺灣桃園地方法院314 n= 314 gc= TYD
8 臺灣新竹地方法院82 n= 82 gc= SCD
9 臺灣苗栗地方法院113 n= 113 gc= MLD
10 臺灣臺中地方法院310 n= 310 gc= TCD
11 臺灣彰化地方法院75 n= 75 gc= CHD
12 臺灣南投地方法院77 n= 77 gc= NTD
13 臺灣雲林地方法院77 n= 77 gc= ULD
14 臺灣嘉義地方法院160 n= 160 gc= CYD
15 臺灣臺南地方法院208 n= 208 gc= TND
16 臺灣高雄地方法院5 n= 5 gc= KSD
17 臺灣橋頭地方法院4 n= 4 gc= CTD
18 臺灣花蓮地方法院112 n= 112 gc= HLD
19 臺灣臺東地方法院65 n= 65 gc= TTD
20 臺灣屏東地方法院12 n= 12 gc= PTD
21 福建金門地方法院1 n= 1 gc= KMD


## 中途失敗的話要跳出框架才可再run

In [61]:
driver.switch_to.parent_frame()

In [384]:
taipei = '//*[@id="collapseGrpCourt"]/div/ul/li[5]/a' #點擊列表第4個法院
driver.find_element_by_xpath(taipei).click()
#driver.find_element_by_xpath(taipei).click() 可能再一次


In [396]:
driver.switch_to.frame("iframe-data") #轉至判決書列表

In [397]:
soup = BeautifulSoup(driver.page_source, 'lxml')
soup

<html lang="zh-TW"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="NO-CACHE" http-equiv="CACHE-CONTROL"/>
<meta content="https://law.judicial.gov.twfblogo.png" property="og:image"/>
<title>
	查詢結果列表 &lt;i class="fa fa-angle-double-right"&gt;&lt;/i&gt;  臺灣臺北地方法院
</title><link href="../css/bootstrap.min.css" rel="stylesheet"/><link href="../css/font-awesome.min.css" rel="stylesheet"/><link href="../css/iframe.css" rel="stylesheet"/>
<script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script src="/js/jquery-3.2.1.min.js" type="text/javascript"></script>
<script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-100643133-1"></script>
<script>
        window.dataLayer = window.dataLayer || [];
        function gtag() { dataLayer.push(arguments); }
        gtag('js', new Date());

        gtag('config', 'UA-100643133-1');
    </script>
<me